# Naive Bayes

In [8]:
import numpy as np
import pandas as pd

In [20]:
def fit(X_train,Y_train):
    result = {}
    result["total_data"] = len(Y_train)
    class_value = set(Y_train)
    for current_class in class_value:
        result[current_class] ={}
        current_class_rows = (Y_train == current_class)#this will give a true false(1 or 0) value np array as o/p
        X_train_current = X_train[current_class_rows]#this will get u all the rows of the current class
        num_features = X_train.shape[1]
        result[current_class]["total_count"] = current_class_row.sum()
        for j in range(1,num_features+1):
            result[current_class][j]={}
            all_possible_values = set(X_train[:,j])
           # result[current_class][j]["distint_features"] = len(all_possible_values)
            for current_value in all_possible_values:
                result[current_class][j][current_value] = (X_train_current[:,j]==current_value).sum()
                
                
    return result
            

In [21]:
def probability(dictionary,x,current_class):
    probability_current_class = dictionary[current_class]["total_count"]/dictionary["total_data"]
    output = output*probabilty_current_class
    num_features = len(dictionary[current_class].keys())-1
    for j in range(1,num_features+1):
        xj = x[j]
        count_current_class_with_value_xj = dictionary[current_class][j][xj]+1
        count_current_class = dictionary[current_class]["total_count"]+ len(dictionary[current_class][j].keys())
        output = output*(count_current_class_with_value_xj/count_current_class)
    return output
    
    
    
    

In [22]:
def predictSinglePoint(dictionary,x):
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        p_current_class = probability(dictionary,x,current_class)
        if (first_run == True or p_current>best_p):
            best_p = p_current
            best_class = current_class
        first_run = False    
    return best_class
            

In [23]:
def predict(dictionary,X_test):
    y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictionary,x)
        y_pred.append(x_class)
    return y_pred
        
        
    